In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import nnls
pd.set_option('display.max_columns', 50)
from IPython.display import display, HTML, Javascript
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
# Comment/uncomment (# prefix) one of the following blocks depending on your source files format

# User-specified files and tab (if MS Excel files)
# Here all tables are in various tabs of the same file but you can specify different files
source_file_format = "xlsx"
signature_file = "./data/signature_415&915_with_correction_factors.xlsx"
signature_tab = "signature_915"
samples_counts_file = "./data/signature_415&915_with_correction_factors.xlsx"
sample_counts_tab = "TLR"
correction_factors_file = "./data/signature_415&915_with_correction_factors.xlsx"
correction_factors_tab = "signature_915_correction_factor"

# User-specified files (if csv files) -- IMPORTANT: Be careful to properly set csv fields separator and decimal separator accordingly to your files content
#source_file_format = "csv"
#csv_separator = ";"
#decimal_separator = ","
#signature_file = "./data/signature_915.csv"
#samples_counts_file = "./data/TLR.csv"
#correction_factors_file = "./data/signature_915_correction_factor.csv"

In [3]:
# 1) Put the content of the ‘signature’ file into a signature dataframe
if source_file_format == "xlsx":
    signature = pd.read_excel(signature_file, signature_tab)
else:
    signature = pd.read_csv(signature_file, sep=csv_separator)
groups = list(signature.columns)[1:]
display(signature)

,Gene,B Memory,B Naive,Basophils LD,CD4+ effector,CD8 activated,MAIT,mDCs,Monocytes C,Monocytes NC+I,Naive T cells,Neutrophils LD,NK,pDCs,Plasmablasts,VD2-,VD2+
0,38596,1,1,0,0,0,0,0,0,0,0,12,0,1,0,0,0
1,AATK,0,0,0,0,0,0,0,2,1,0,18,1,0,0,0,0
2,ABC7-481722F1.1,0,0,0,0,0,0,1,5,2,0,17,0,0,0,0,0
3,ABCB4,9,41,0,0,1,0,5,0,0,0,0,1,1,0,0,0
4,ABCC3,4,6,5,3,2,2,4,14,65,4,3,4,3,1,2,1
5,AC002480.4,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,AC002480.5,2,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,AC002511.3,0,0,4,0,0,0,0,3,1,0,27,0,0,0,0,0
8,AC007381.3,1,2,0,0,0,0,0,0,0,0,0,0,16,0,0,0
9,AC008074.5,0,0,0,0,0,0,0,0,0,0,14,0,0,0,0,0


In [4]:
# 2) Compute weighting: for each row of the signature dataframe, weight = 1 / max
weighting = []
for index, row in signature.iterrows():
    current_row = []
    for header in groups:
        current_row.append(row[header])
    weighting.append(1/max(current_row))
print(weighting)

[0.08333333333333333, 0.05555555555555555, 0.058823529411764705, 0.024390243902439025, 0.015384615384615385, 0.1, 0.0625, 0.037037037037037035, 0.0625, 0.07142857142857142, 0.07692307692307693, 0.04, 0.04, 0.022727272727272728, 0.04, 0.01282051282051282, 0.02040816326530612, 0.0026595744680851063, 0.05, 0.058823529411764705, 0.0043859649122807015, 0.02702702702702703, 0.03333333333333333, 0.016129032258064516, 0.05555555555555555, 0.006097560975609756, 0.0006993006993006993, 0.024390243902439025, 0.029411764705882353, 0.002544529262086514, 0.018518518518518517, 0.07692307692307693, 0.08333333333333333, 0.0017271157167530224, 0.004739336492890996, 0.0625, 0.008, 0.003125, 0.016129032258064516, 0.005263157894736842, 0.014925373134328358, 0.001440922190201729, 0.038461538461538464, 0.038461538461538464, 0.1, 0.05555555555555555, 0.0026954177897574125, 0.05555555555555555, 0.06666666666666667, 0.07692307692307693, 0.00847457627118644, 0.04, 0.0014144271570014145, 0.0009606147934678194, 0.0

In [5]:
# 3) Put the content of the ‘samples’ file to be deconvoluted into a sample dataframe
# This file have to contain genes in rows and samples in columns.
# First colum contains gene names
if source_file_format == "xlsx":
    samples_counts = pd.read_excel(samples_counts_file, sample_counts_tab)
else:
    samples_counts = pd.read_csv(samples_counts_file, sep=csv_separator, decimal=decimal_separator)
samples = list(samples_counts.columns)[1:]

# 4) Merge both samples and signature dataframes keeping only common genes, result being merged_table dataframe
merged_table = pd.merge(signature, samples_counts, how='inner', on=['Gene'])
display(merged_table)

,Gene,B Memory,B Naive,Basophils LD,CD4+ effector,CD8 activated,MAIT,mDCs,Monocytes C,Monocytes NC+I,Naive T cells,Neutrophils LD,NK,pDCs,Plasmablasts,VD2-,VD2+,A1_ 24h_ BNT1_ 1,A1_ 24h_ BNT1_ 100,A1_ 24h_ BNT1_ 1000,A1_ 24h_ DMSO_ Veh,A1_ 24h_ MOTO_ 1,A1_ 24h_ MOTO_ 100,A1_ 24h_ MOTO_ 1000,A1_ 24h_ NVS_ 1,...,B2_ 2h_ RA389_ 1,B2_ 2h_ RA389_ 100,B2_ 2h_ RA389_ 1000,B2_ 2h_ RA97B_ 1,B2_ 2h_ RA97B_ 100,B2_ 2h_ RA97B_ 1000,B2_ 6h_ BNT1_ 1,B2_ 6h_ BNT1_ 100,B2_ 6h_ BNT1_ 1000,B2_ 6h_ DMSO_ Veh,B2_ 6h_ MOTO_ 1,B2_ 6h_ MOTO_ 100,B2_ 6h_ MOTO_ 1000,B2_ 6h_ NVS_ 1,B2_ 6h_ NVS_ 100,B2_ 6h_ NVS_ 1000,B2_ 6h_ R848_ 1,B2_ 6h_ R848_ 100,B2_ 6h_ R848_ 1000,B2_ 6h_ RA389_ 1,B2_ 6h_ RA389_ 100,B2_ 6h_ RA389_ 1000,B2_ 6h_ RA97B_ 1,B2_ 6h_ RA97B_ 100,B2_ 6h_ RA97B_ 1000
0,AATK,0,0,0,0,0,0,0,2,1,0,18,1,0,0,0,0,30.1877,13.7443,7.4445,21.1753,32.6298,40.0548,6.4657,7.1996,...,101.5625,84.9854,67.2501,97.2506,110.9209,100.4686,181.5845,137.1388,45.2347,132.4225,151.4649,28.5706,10.5535,156.9964,48.0176,50.9834,109.0754,108.4192,25.2855,148.5827,84.5774,47.3708,239.9675,90.3909,56.9016
1,ABCB4,9,41,0,0,1,0,5,0,0,0,0,1,1,0,0,0,6.4229,1.8085,2.4815,11.2578,17.7133,5.0979,9.2367,4.1141,...,30.4377,29.6774,26.3620,18.7509,17.3037,34.0048,26.6476,22.7162,16.2381,30.8788,37.0935,21.4280,38.4146,29.4368,12.9278,10.4581,26.8493,30.5545,6.9939,26.0020,20.6286,8.1988,26.0689,24.2030,11.3803
2,ABCC3,4,6,5,3,2,2,4,14,65,4,3,4,3,1,2,1,71.2944,6.1488,4.9630,102.1239,98.8218,29.8590,3.6947,23.6559,...,48.1413,33.7244,47.8821,54.3459,66.5525,42.5060,28.9317,15.9855,20.8776,19.5961,12.3645,16.2333,25.3283,22.2412,14.7747,13.0727,6.7123,17.7413,16.6776,16.7155,13.0648,16.3976,25.4005,20.7455,12.8029
3,ACSL1,21,12,57,8,10,7,28,145,82,5,1430,12,9,3,9,8,486.8571,4104.4834,3839.6846,403.1347,426.0526,3134.4702,2803.9688,3647.1245,...,3349.6992,3457.4216,5219.6821,3574.1199,4128.0308,5094.9189,15690.4980,14668.3838,10796.0137,13502.9336,10383.0752,9377.0029,824.8578,9962.0762,5642.0737,4137.4961,10135.6172,11055.7979,3523.8257,12518.5537,13693.9805,8478.4580,13424.1455,18974.1934,11609.3564
4,ADAM12,0,0,0,6,0,41,0,1,0,0,0,0,0,0,0,0,1.2846,2.8935,1.6543,4.2887,6.7590,8.7392,1.8473,3.0856,...,1.5529,0.6745,1.0760,2.2247,0.8874,0.7728,1.5227,2.1034,1.1599,2.3753,3.0911,2.5973,1.2664,1.3083,5.5405,1.3073,1.6781,1.9713,1.6140,1.8573,1.3752,0.9110,2.0053,2.4697,4.2676
5,ADAMTS1,0,2,0,0,4,1,1,1,1,1,0,34,0,0,7,4,17.3419,0.7234,3.3086,16.6186,13.9842,1.4565,7.0815,0.0000,...,9.9388,6.0704,5.3800,5.4028,8.8737,6.9555,8.7556,3.7860,2.3197,2.9691,2.0607,1.2987,4.6435,5.2332,1.8468,2.6145,8.3904,2.9569,1.0760,5.5718,6.1886,3.6439,7.3528,3.4576,1.4225
6,ADM,2,2,0,0,0,0,2,36,19,0,393,0,0,1,0,0,77.7173,404.0102,305.2227,71.2991,80.4093,279.6553,396.8723,371.2950,...,414.3253,561.1734,1171.2274,407.7535,633.5800,769.7443,783.0593,1032.7478,1163.3435,912.1116,729.5046,1033.0872,155.7690,630.6022,1229.9905,681.0854,481.6096,940.2899,728.4366,664.4431,1301.6674,1160.5840,913.7483,1646.3004,1372.7520
7,AEBP1,2,12,1,3,1,0,0,0,0,11,0,1,54,0,1,2,13.4881,5.7871,9.0988,15.0103,12.5858,14.5654,8.9289,6.1711,...,5.9012,6.0704,4.3040,2.8603,6.2116,2.3185,7.2329,3.7860,4.6395,4.7506,6.1822,3.2467,4.2214,6.5415,7.3873,2.6145,1.6781,3.9425,4.3039,3.2502,6.1886,7.2878,11.3634,5.9273,7.1127
8,AIF1,12,18,1,3,0,0,210,247,579,21,226,3,16,1,0,2,3494.7087,1487.6401,785.8036,3813.6970,4498.0249,4049.1763,217.3715,760.0747,...,2198.3469,2496.2773,2104.1206,2436.9866,2399.4399,2020.5789,673.4234,530.8870,274.8878,585.5092,705.8060,346.0939,64.5871,578.2701,317.6552,220.9279,533.6302,508.5844,188.8340,699.7315,341.0602,225.0112,697.1759,555.1880,352.7902
9,AIM2,211,45,8,6,4,3,5,11,11,1,27,2,0,22,2,3,82.8556,1189.6057,837.0876,60.3094,59.8991,178.4259,767.8817,859.8409,...,79.2001,111.9649,140.9562,120.4510,110.4772,145.6795,174.7322,273.4363,724.9150,186.4603,200.9229,725.3038,522.6069,161.5755,419.2310,529.4426,223.1850,295.6886,498.1775,204.3012,558.3486,569.3603,98.2597,291.9183,419.6496


In [6]:
# 5) Apply weighting: Multiply each value of each row by the weight from the weighting list (index of the weight is equal to index of the row)
# Result is a weighted merged signature/samples dataframe called weighted_merged_table
col_headers = list(merged_table.columns)
weighted_merged_table = pd.DataFrame(columns=col_headers)
for index, row in merged_table.iterrows():
    weighted_row = {}
    current_cell_is_gene = True # First iteration is on the column that contains genes
    for header in col_headers:
        if current_cell_is_gene:
            current_cell_is_gene = False
            weighted_row[header] = row[header]
            continue
        weighted_row[header] = row[header] * weighting[index]
    weighted_merged_table = weighted_merged_table.append(weighted_row, ignore_index=True)
display(weighted_merged_table)

,Gene,B Memory,B Naive,Basophils LD,CD4+ effector,CD8 activated,MAIT,mDCs,Monocytes C,Monocytes NC+I,Naive T cells,Neutrophils LD,NK,pDCs,Plasmablasts,VD2-,VD2+,A1_ 24h_ BNT1_ 1,A1_ 24h_ BNT1_ 100,A1_ 24h_ BNT1_ 1000,A1_ 24h_ DMSO_ Veh,A1_ 24h_ MOTO_ 1,A1_ 24h_ MOTO_ 100,A1_ 24h_ MOTO_ 1000,A1_ 24h_ NVS_ 1,...,B2_ 2h_ RA389_ 1,B2_ 2h_ RA389_ 100,B2_ 2h_ RA389_ 1000,B2_ 2h_ RA97B_ 1,B2_ 2h_ RA97B_ 100,B2_ 2h_ RA97B_ 1000,B2_ 6h_ BNT1_ 1,B2_ 6h_ BNT1_ 100,B2_ 6h_ BNT1_ 1000,B2_ 6h_ DMSO_ Veh,B2_ 6h_ MOTO_ 1,B2_ 6h_ MOTO_ 100,B2_ 6h_ MOTO_ 1000,B2_ 6h_ NVS_ 1,B2_ 6h_ NVS_ 100,B2_ 6h_ NVS_ 1000,B2_ 6h_ R848_ 1,B2_ 6h_ R848_ 100,B2_ 6h_ R848_ 1000,B2_ 6h_ RA389_ 1,B2_ 6h_ RA389_ 100,B2_ 6h_ RA389_ 1000,B2_ 6h_ RA97B_ 1,B2_ 6h_ RA97B_ 100,B2_ 6h_ RA97B_ 1000
0,AATK,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.083333,0.000000,1.500000,0.083333,0.000000,0.000000,0.000000,0.000000,2.515642,1.145358,0.620375,1.764608,2.719150,3.337900,0.538808,0.599967,...,8.463542,7.082117,5.604175,8.104217,9.243408,8.372383,15.132042,11.428233,3.769558,11.035208,12.622075,2.380883,0.879458,13.083033,4.001467,4.248617,9.089617,9.034933,2.107125,12.381892,7.048117,3.947567,19.997292,7.532575,4.741800
1,ABCB4,0.500000,2.277778,0.000000,0.000000,0.055556,0.000000,0.277778,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.356828,0.100472,0.137861,0.625433,0.984072,0.283217,0.513150,0.228561,...,1.690983,1.648744,1.464556,1.041717,0.961317,1.889156,1.480422,1.262011,0.902117,1.715489,2.060750,1.190444,2.134144,1.635378,0.718211,0.581006,1.491628,1.697472,0.388550,1.444556,1.146033,0.455489,1.448272,1.344611,0.632239
2,ABCC3,0.235294,0.352941,0.294118,0.176471,0.117647,0.117647,0.235294,0.823529,3.823529,0.235294,0.176471,0.235294,0.176471,0.058824,0.117647,0.058824,4.193788,0.361694,0.291941,6.007288,5.813047,1.756412,0.217335,1.391524,...,2.831841,1.983788,2.816594,3.196818,3.914853,2.500353,1.701865,0.940324,1.228094,1.152712,0.727324,0.954900,1.489900,1.308306,0.869100,0.768982,0.394841,1.043606,0.981035,0.983265,0.768518,0.964565,1.494147,1.220324,0.753112
3,ACSL1,0.512195,0.292683,1.390244,0.195122,0.243902,0.170732,0.682927,3.536585,2.000000,0.121951,34.878049,0.292683,0.219512,0.073171,0.219512,0.195122,11.874563,100.109351,93.650844,9.832554,10.391527,76.450493,68.389483,88.954256,...,81.699980,84.327356,127.309320,87.173656,100.683678,124.266315,382.695073,357.765459,263.317407,329.339844,253.245737,228.707388,20.118483,242.977468,137.611554,100.914539,247.210176,269.653607,85.946968,305.330578,333.999524,206.791659,327.418183,462.785205,283.155034
4,ADAM12,0.000000,0.000000,0.000000,0.092308,0.000000,0.630769,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019763,0.044515,0.025451,0.065980,0.103985,0.134449,0.028420,0.047471,...,0.023891,0.010377,0.016554,0.034226,0.013652,0.011889,0.023426,0.032360,0.017845,0.036543,0.047555,0.039958,0.019483,0.020128,0.085238,0.020112,0.025817,0.030328,0.024831,0.028574,0.021157,0.014015,0.030851,0.037995,0.065655
5,ADAMTS1,0.000000,0.200000,0.000000,0.000000,0.400000,0.100000,0.100000,0.100000,0.100000,0.100000,0.000000,3.400000,0.000000,0.000000,0.700000,0.400000,1.734190,0.072340,0.330860,1.661860,1.398420,0.145650,0.708150,0.000000,...,0.993880,0.607040,0.538000,0.540280,0.887370,0.695550,0.875560,0.378600,0.231970,0.296910,0.206070,0.129870,0.464350,0.523320,0.184680,0.261450,0.839040,0.295690,0.107600,0.557180,0.618860,0.364390,0.735280,0.345760,0.142250
6,ADM,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,0.125000,2.250000,1.187500,0.000000,24.562500,0.000000,0.000000,0.062500,0.000000,0.000000,4.857331,25.250637,19.076419,4.456194,5.025581,17.478456,24.804519,23.205938,...,25.895331,35.073338,73.201712,25.484594,39.598750,48.109019,48.941206,64.546738,72.708969,57.006975,45.594037,64.567950,9.735563,39.412638,76.874406,42.567838,30.100600,58.768119,45.527287,41.527694,81.354213,72.536500,57.10

In [7]:
# 6) Run nnls regression of each sample on the signature to get one estimate per cell group for each sample
weighted_merged_table_values = weighted_merged_table.iloc[:,1:].values
weighted_signature_group_values = weighted_merged_table_values[:, :len(groups)]
x = np.zeros((len(groups), len(samples)))
for i in range(0, len(samples)):
    weighted_sample_column_values = weighted_merged_table_values[:, i+len(groups)]
    x[:,i] = nnls(weighted_signature_group_values, weighted_sample_column_values)[0] * 100
nnls_result = x.T
print(nnls_result)
np.savetxt('./data/proportions.csv', nnls_result, delimiter=',')

[[   0.            0.            0.         ...,    0.          111.13298098
     0.        ]
 [   0.            0.            2.5095664  ...,   14.61305995    0.
   110.74541455]
 [  54.96865617    2.08106029    1.19732826 ...,    0.            0.
   151.78255926]
 ..., 
 [   0.           38.91294627    0.         ...,    4.67531157    0.
    14.6430445 ]
 [   0.            0.            0.         ...,    9.4926082     0.
   183.33214537]
 [   0.            0.79659617    0.         ...,   10.76449507    0.
   184.4112103 ]]


In [8]:
# 7) Put the content of the ‘correction factors’ file into a list and multiply each estimate by the corresponding cell group correction factor
if source_file_format == "xlsx":
    correction_factors_table = pd.read_excel(correction_factors_file, correction_factors_tab, index_col=0)
else:
    correction_factors_table = pd.read_csv(correction_factors_file, sep=csv_separator, decimal=decimal_separator, index_col=0)
correction_factors = correction_factors_table.values
nnls_result[0]
result = {}
for sample in range(len(nnls_result)):
    current_sample = samples[sample]
    result[current_sample] = []
    for group in range(len(nnls_result[sample])):
        result[current_sample].append(nnls_result[sample][group]*correction_factors[group][0])
result_table = pd.DataFrame.from_dict(result, orient='index', columns=groups)
display(result_table)

,B Memory,B Naive,Basophils LD,CD4+ effector,CD8 activated,MAIT,mDCs,Monocytes C,Monocytes NC+I,Naive T cells,Neutrophils LD,NK,pDCs,Plasmablasts,VD2-,VD2+
A1_ 24h_ BNT1_ 1,0.000000,0.000000,0.000000,0.000000,23.093848,104.406071,51.856054,207.196758,0.000000,105.147920,0.000000,154.256184,0.000000,0.000000,105.444453,0.000000
A1_ 24h_ BNT1_ 100,0.000000,0.000000,2.842553,16.084674,0.000000,0.000000,24.312966,12.181480,31.140160,408.786510,4.038601,157.090900,0.000000,1.104051,0.000000,109.080556
A1_ 24h_ BNT1_ 1000,32.981194,2.289166,1.356198,0.000000,0.000000,0.000000,14.289300,0.000000,21.244324,562.961087,9.602940,139.914264,0.000000,0.000000,0.000000,149.500781
A1_ 24h_ DMSO_ Veh,0.000000,0.000000,0.000000,0.000000,17.879981,94.042093,45.956904,207.930897,0.000000,93.844120,0.000000,130.046767,0.000000,0.000000,123.851580,0.000000
A1_ 24h_ MOTO_ 1,0.000000,0.000000,0.000000,0.000000,22.678464,89.401961,59.125176,223.605673,0.000000,94.346330,0.000000,154.074803,0.000000,0.000000,74.746032,0.000000
A1_ 24h_ MOTO_ 100,65.106153,2.078067,0.000000,0.000000,38.131165,0.000000,1.186168,138.155991,42.050381,130.172486,0.000000,82.624832,0.000000,0.000000,309.177401,89.058339
A1_ 24h_ MOTO_ 1000,52.432050,4.419553,0.529897,0.000000,10.749640,0.000000,2.385950,0.000000,20.467199,554.367197,19.273208,163.769708,0.000000,0.000000,0.000000,366.108489
A1_ 24h_ NVS_ 1,2.657348,0.000000,2.545276,0.000000,0.000000,0.000000,31.195065,0.304769,10.098291,449.919505,4.104509,134.117516,0.000000,0.954588,0.000000,155.662420
A1_ 24h_ NVS_ 100,37.631509,0.000000,0.000000,0.000000,50.090499,0.000000,12.438113,86.230835,21.688596,266.873740,0.000000,163.532283,0.000000,0.000000,0.000000,138.745665
A1_ 24h_ NVS_ 1000,70.297936,0.000000,0.000000,0.000000,88.437263,1.501383,2.386237,79.169532,29.341476,244.166998,10.628368,167.806874,0.000000,0.000000,0.000000,117.522786


In [ ]:
def csv_download(table, table_name):
    js_download = """
    var csv = '%s';

    var filename = '%s.csv';
    var blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
    if (navigator.msSaveBlob) { // IE 10+
        navigator.msSaveBlob(blob, filename);
    } else {
        var link = document.createElement("a");
        if (link.download !== undefined) { // feature detection
            // Browsers that support HTML5 download attribute
            var url = URL.createObjectURL(blob);
            link.setAttribute("href", url);
            link.setAttribute("download", filename);
            link.style.visibility = 'hidden';
            document.body.appendChild(link);
            link.click();
            document.body.removeChild(link);
        }
    }
    """ % (table.to_csv(index=True).replace('\n','\\n').replace("'","\'"), table_name)

    return Javascript(js_download)

csv_download(result_table, "deconvolution_result_table")